In [3]:
import pandas as pd

# Définition des chemins des fichiers
file_2023 = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/caract-2023.csv"
file_2022 = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/carcteristiques-2022.csv"

# Charger les fichiers avec le bon séparateur
df_2023 = pd.read_csv(file_2023, sep=";", dtype=str)
df_2022 = pd.read_csv(file_2022, sep=";", dtype=str)

# Renommer la colonne 'Accident_Id' en 'Num_Acc' dans le fichier 2022
df_2022.rename(columns={"Accident_Id": "Num_Acc"}, inplace=True)

# Vérifier si les colonnes sont identiques avant concaténation
if list(df_2023.columns) == list(df_2022.columns):
    # Concaténer les deux DataFrames
    df_2022_2023 = pd.concat([df_2023, df_2022], ignore_index=True)
    
    # Sauvegarder le DataFrame final en CSV si besoin
    df_2022_2023.to_csv("/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/concatene_2022_2023.csv", index=False, sep=";")
    
    print


In [ ]:
import pandas as pd

# Charger le fichier concaténé
file_path = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/concatene_2022_2023.csv"
df = pd.read_csv(file_path, sep=";", dtype=str)

# Filtrer les lignes où dep == 77
df_filtered = df[df["dep"] == "77"]

# Compter le nombre d'accidents par commune et par jour
df_grouped = df_filtered.groupby(["jour", "mois", "an", "dep", "com"]).size().reset_index(name="nombre_d_accidents")


  jour mois    an dep    com  nombre_d_accidents
0    1   01  2022  77  77231                   1
1    1   01  2022  77  77437                   1
2    1   01  2023  77  77182                   1
3    1   01  2023  77  77237                   1
4    1   01  2023  77  77384                   1


In [5]:
df_grouped

,jour,mois,an,dep,com,nombre_d_accidents
0,1,01,2022,77,77231,1
1,1,01,2022,77,77437,1
2,1,01,2023,77,77182,1
3,1,01,2023,77,77237,1
4,1,01,2023,77,77384,1
...,...,...,...,...,...,...
1697,9,12,2022,77,77142,1
1698,9,12,2023,77,77122,1
1699,9,12,2023,77,77209,1
1700,9,12,2023,77,77251,1


In [9]:
df_grouped.to_csv("/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/grouped.csv", index=False, sep=";")

In [8]:
import requests
import pandas as pd
import os

# URL de l'API gouvernementale
url = "https://geo.api.gouv.fr/departements/77/communes"

# Paramètres de la requête
params = {
    "fields": "nom,codesPostaux",
    "format": "json"
}

try:
    # Requête GET vers l'API
    response = requests.get(url, params=params)
    response.raise_for_status()  # Vérifie si la requête a réussi

    # Récupération des données JSON
    communes = response.json()

    # Extraction des noms de communes et codes postaux
    data = []
    for commune in communes:
        nom_commune = commune["nom"]
        codes_postaux = ", ".join(commune["codesPostaux"])
        data.append([nom_commune, codes_postaux])

    # Création du DataFrame
    df_communes = pd.DataFrame(data, columns=["Commune", "Codes Postaux"])

    # Définition du chemin d'enregistrement
    save_dir = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet"
    save_path = os.path.join(save_dir, "communes_77.csv")

    # Vérifier si le dossier existe, sinon le créer
    os.makedirs(save_dir, exist_ok=True)

    # Enregistrement du fichier CSV
    df_communes.to_csv(save_path, index=False, encoding="utf-8")

    print(f"✅ Fichier enregistré avec succès : {save_path}")

except requests.exceptions.RequestException as e:
    print(f"❌ Erreur lors de la requête API : {e}")
except Exception as e:
    print(f"❌ Une erreur est survenue : {e}")


✅ Fichier enregistré avec succès : /Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/communes_77.csv


In [ ]:
import pandas as pd
from itertools import product

# 📂 Définition des chemins des fichiers
path_communes = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/communes_77.csv"
path_accidents = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/grouped.csv"
path_feries = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/jours_feries_metropole.csv"
path_vacances = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/vacances_scolaires_2017_2025.csv"
path_output = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/accidents_completes.csv"

# 🏙 Charger la liste des communes
df_communes = pd.read_csv(path_communes)
df_communes.rename(columns={"Codes Postaux": "com"}, inplace=True)

# 🚗 Charger les accidents
df_accidents = pd.read_csv(path_accidents, sep=";")

# 📅 Charger les jours fériés
df_feries = pd.read_csv(path_feries, sep=",")
df_feries["Date"] = pd.to_datetime(df_feries["Date"], format="%d/%m/%Y")

# 🏫 Charger les vacances scolaires
df_vacances = pd.read_csv(path_vacances, sep=",")
df_vacances["debut"] = pd.to_datetime(df_vacances["debut"], format="%d/%m/%Y")
df_vacances["fin"] = pd.to_datetime(df_vacances["fin"], format="%d/%m/%Y")

# 📆 Générer toutes les dates de 2022 à 2023
dates = pd.date_range(start="2022-01-01", end="2023-12-31")
df_dates = pd.DataFrame({"date": dates})
df_dates["jour"] = df_dates["date"].dt.day
df_dates["mois"] = df_dates["date"].dt.month.astype(str).str.zfill(2)  
df_dates["an"] = df_dates["date"].dt.year

# 📌 Marquer les jours fériés
df_dates["jour_ferie"] = df_dates["date"].isin(df_feries["Date"])

# 🏫 Marquer les jours de vacances pour chaque zone
for zone in ["Zone A", "Zone B", "Zone C"]:
    df_dates[f"vacances_{zone.replace(' ', '_')}"] = df_dates["date"].apply(
        lambda d: any((d >= row["debut"]) and (d <= row["fin"]) and (zone in row["zone"]) for _, row in df_vacances.iterrows())
    )

df_dates.drop(columns=["date"], inplace=True)

# 🏘 Créer toutes les combinaisons entre dates et communes
df_complet = pd.DataFrame(product(df_dates.itertuples(index=False), df_communes["com"]),
                          columns=["date_info", "com"])

# 🧩 Séparer les valeurs de date_info en colonnes
df_complet[["jour", "mois", "an", "jour_ferie", "vacances_Zone_A", "vacances_Zone_B", "vacances_Zone_C"]] = pd.DataFrame(df_complet["date_info"].tolist(), index=df_complet.index)
df_complet.drop(columns=["date_info"], inplace=True)

# 🏢 Ajouter le département 77
df_complet["dep"] = 77

# 📊 Assurer que les types sont bien les mêmes
df_accidents["com"] = df_accidents["com"].astype(str)
df_complet["com"] = df_complet["com"].astype(str)

df_accidents["mois"] = df_accidents["mois"].astype(str).str.zfill(2)  
df_complet["mois"] = df_complet["mois"].astype(str).str.zfill(2)  

# 🔗 Joindre avec les accidents pour récupérer les données existantes
df_final = df_complet.merge(df_accidents, on=["jour", "mois", "an", "dep", "com"], how="left")

# 🔄 Remplacer les NaN par 0 pour les jours sans accident
df_final["nombre_d_accidents"] = df_final["nombre_d_accidents"].fillna(0).astype(int)

# 💾 Sauvegarder le fichier final
df_final.to_csv(path_output, index=False)

print(f"✅ Fichier généré avec succès : {path_output}")


✅ Fichier généré avec succès : /Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/accidents_completes.csv


In [21]:
import pandas as pd

# 📌 Chemin du fichier météo
path_meteo = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/git/jedha-lead/data/meteo_celsius_Quiers_2022_2023.csv"

# 📥 Charger les données météo
df_meteo = pd.read_csv(path_meteo)

# 🛠️ Nettoyer la colonne dt_iso pour enlever le décalage horaire et "UTC"
df_meteo["dt_iso"] = df_meteo["dt_iso"].str.extract(r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})")

# 🗓️ Convertir en format date (on garde seulement YYYY-MM-DD)
df_meteo["date"] = pd.to_datetime(df_meteo["dt_iso"]).dt.date

# 📊 Agréger les données par jour
df_meteo_daily = df_meteo.groupby("date").agg({
    "temp": "mean",
    "feels_like": "mean",
    "temp_min": "min",
    "temp_max": "max",
    "pressure": "mean",
    "humidity": "mean",
    "wind_speed": "mean",
    "clouds_all": "mean"
}).reset_index()

# 💾 Sauvegarde avec séparation par virgule et encodage UTF-8
output_path = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/meteo_aggree_par_jour.csv"
df_meteo_daily.to_csv(output_path, index=False, sep=",", encoding="utf-8")

print(f"✅ Fichier météo agrégé enregistré à : {output_path}")


✅ Fichier météo agrégé enregistré à : /Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/meteo_aggree_par_jour.csv


In [23]:
import pandas as pd

# 📌 Chemins des fichiers
path_accidents = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/accidents_completes.csv"
path_meteo = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/meteo_aggree_par_jour.csv"
save_path = "/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/accidents_77_final.csv"

# 📥 Charger les données d'accidents
df_accidents = pd.read_csv(path_accidents, sep=";")

# 🛠 Vérifier la présence des colonnes essentielles
assert all(col in df_accidents.columns for col in ["jour", "mois", "an"]), "Colonnes 'jour', 'mois', 'an' manquantes !"

# 🔢 Convertir "jour", "mois", "an" en entiers
df_accidents["jour"] = df_accidents["jour"].astype(int)
df_accidents["mois"] = df_accidents["mois"].astype(int)
df_accidents["an"] = df_accidents["an"].astype(int)

# 🗓️ Créer une colonne "date" au format YYYY-MM-DD
df_accidents["date"] = pd.to_datetime(df_accidents[["an", "mois", "jour"]].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d')

# 📥 Charger les données météo
df_meteo = pd.read_csv(path_meteo)

# 🗓️ Convertir "date" en datetime
df_meteo["date"] = pd.to_datetime(df_meteo["date"])

# 🎯 Ne garder que les colonnes météo nécessaires
cols_meteo = ["date", "temp", "feels_like", "temp_min", "temp_max", "pressure", "humidity", "wind_speed", "clouds_all"]
df_meteo = df_meteo[cols_meteo]

# 📊 Fusionner les datasets sur la colonne "date"
df_final = df_accidents.merge(df_meteo, on="date", how="left")

# 💾 Enregistrer en UTF-8 avec le bon séparateur
df_final.to_csv(save_path, index=False, sep=",", encoding="utf-8")

print(f"✅ Fichier enregistré avec succès à : {save_path}")


✅ Fichier enregistré avec succès à : /Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/accidents_77_final.csv


In [25]:
df_final_77 = pd.read_csv('/Users/maurice/Documents/jedha/jedha/lead/00_projet_lead/datasets_projet/accidents_77_final.csv')

In [19]:
df_final_77

,com,jour,mois,an,jour_ferie,vacances_Zone_A,vacances_Zone_B,vacances_Zone_C,dep,nombre_d_accidents,...,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id
0,77760,1,1,2022,True,True,True,True,77,0,...,8.88,8.19,7.81,10.96,1027,98,1.67,198,64,803
1,77120,1,1,2022,True,True,True,True,77,0,...,8.88,8.19,7.81,10.96,1027,98,1.67,198,64,803
2,77760,1,1,2022,True,True,True,True,77,0,...,8.88,8.19,7.81,10.96,1027,98,1.67,198,64,803
3,77390,1,1,2022,True,True,True,True,77,0,...,8.88,8.19,7.81,10.96,1027,98,1.67,198,64,803
4,77410,1,1,2022,True,True,True,True,77,0,...,8.88,8.19,7.81,10.96,1027,98,1.67,198,64,803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370105,77560,31,12,2023,False,True,True,True,77,0,...,9.17,6.01,8.38,9.84,1005,89,6.55,184,100,804
370106,77940,31,12,2023,False,True,True,True,77,0,...,9.17,6.01,8.38,9.84,1005,89,6.55,184,100,804
370107,77160,31,12,2023,False,True,True,True,77,0,...,9.17,6.01,8.38,9.84,1005,89,6.55,184,100,804
370108,77870,31,12,2023,False,True,True,True,77,0,...,9.17,6.01,8.38,9.84,1005,89,6.55,184,100,804
